In [2]:
import sys
sys.path.append('../consensus/')
import consensus as Con

con = Con.Consensus()


In [3]:
import numpy as np
import warnings

warnings.filterwarnings('ignore')

def sigmoid(x):
    # exp = "[1 / (1 + np.exp(-value)) for value in x]"
    # return con.evaluate(exp, values={'nenc' : {'np' : np}, 'enc' : {'x' : x}})
    return 1/(1+np.exp(-x))

class LogisticRegression():

    def __init__(self, lr=0.001, n_iters=1000):
        self.lr = lr
        self.n_iters = n_iters
        self.weights = None
        self.bias = None

    def fit(self, X, y, weights, bias):
        n_samples, n_features = X.shape
        if weights != None:
            self.weights = weights
        else:
            self.weights = np.zeros(n_features)
        if bias != None:
            self.bias = bias
        else:
            self.bias = 0
        self.weights = con.encrypt(self.weights, mod=1)
        self.bias = con.encrypt(self.bias, mod = 1) 
        # print(self.weights, self.bias)
        for _ in range(self.n_iters):
            linear_pred = "np.dot(X, weights) + bias"
            linear_pred = con.evaluate(linear_pred, values={'nenc' : {'X' : X, 'np' : np}, 'enc': {'weights' : self.weights, 'bias' : self.bias}})

            # predictions = sigmoid(linear_pred)
            # predictions = con.evaluate(linear_pred, v)
            
            sigmoid = "np.maximum(0, x)"
            predictions = con.evaluate(sigmoid, values = {'nenc' : {'np' : np}, 'enc': {'x' : linear_pred}})
            # predictions = sigmoid(linear_pred)

            n = (1 / n_samples)

            dw = "n * np.dot(X, (predictions - y))"
            dw = con.evaluate(dw, values={'nenc' : {'n' : n, 'X' : X.T, 'y' : y, 'np' : np}, 'enc' : { 'predictions' : predictions}})
            db = "n * np.sum(predictions-y)"
            db = con.evaluate(db, values={'nenc' : {'n' : n, 'y' : y, 'np' : np}, 'enc' : { 'predictions' : predictions}})
            exp = "[w - (value['lr'] * d) for w, d in zip(value['weights'], value['dw'])]"
            self.weights = con.evaluate(exp, values={'nenc' : {'lr' : self.lr}, 'enc' : {'weights' : self.weights, 'dw' : dw}}, cal="norm")
        #     self.bias = self.bias - self.lr*db
            # print("self.bias", self.bias)
            # print("self.db", db)
            exp = "bias - lr * db"
            self.bias = con.evaluate(exp, values={'nenc' : {'lr' : self.lr}, 'enc' : {'bias' : self.bias, 'db' : db}})


    def predict(self, X):
        linear_pred = "np.dot(X, weights) + bias"
        linear_pred = con.evaluate(linear_pred, values={'nenc' : {'X' : X, 'np' : np}, 'enc': {'weights' : self.weights, 'bias' : self.bias}})

        # y_pred = sigmoid(linear_pred)
        sigmoid = "np.maximum(0, x)"
        y_pred = con.evaluate(sigmoid, values = {'nenc' : {'np' : np}, 'enc': {'x' : linear_pred}})

        # y_pred = sigmoid(linear_pred)

        # class_pred = [0 if y<=0.5 else 1 for y in y_pred]
        exp = "[0 if y<=0.5 else 1 for y in y_pred]"
        class_pred = con.evaluate(exp, values={'enc' : {'y_pred' : y_pred}})
        return class_pred

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score




bc = datasets.load_breast_cancer()
# bc.data = np.array([[f"{val:.9f}" for val in row] for row in bc.data])

# print(bc.data[:5])

X, y = bc.data, bc.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

clf = LogisticRegression(lr=0.01)
clf.fit(X_train,y_train, None, None)
y_pred = clf.predict(X_test)

def accuracy(y_pred, y_test):
    exp = "np.sum(y_pred==y_test)/len(y_test)"
    return con.evaluate(exp, values={'nenc' : {'y_test' : y_test, 'np' : np}, 'enc' : {'y_pred' : y_pred}})

acc = accuracy(y_pred, y_test)
print("Accuracy : ", acc)
print("Decrypted Accuracy", con.decrypt(acc))
# print("Overall Accuracy : ", accuracy_score(y_test, y_pred))


Accuracy :  by..y93g
Decrypted Accuracy 0.39
